In [1]:
import sys
import os
from glob import glob
sys.path.append("../")
from NEATModels import NEATStaticDetection, nets
from NEATModels.Staticconfig  import StaticNeatConfig
from NEATUtils import helpers
from NEATUtils.helpers import save_json, load_json
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

Using TensorFlow backend.


In [2]:
#Input the training directory containing training image patches and labels 
TrainDataDir = '/data/u934/service_imagerie/v_kapoor/FinalONEATTraining/StaticCenterTrainData/'
#Directory containing the train category json, model parameter and weight vector file
ModelDir = '/data/u934/service_imagerie/v_kapoor/FinalONEATTraining/NEATModel/'
#Model name based on wether it is residual or sequntial ONEAT network
Model_Name = 'ORStaticONEAT.h5'
StaticCategoriesJson = ModelDir + 'StaticCategories.json'
KeyCategories = load_json(StaticCategoriesJson)

StaticCordJson = ModelDir + 'StaticCord.json'
KeyCord = load_json(StaticCordJson)

In [3]:
#Neural network parameters
#For ORNET use residual = True and for OSNET use residual = False
residual = True
#NUmber of starting convolutional filters, is doubled down with increasing depth
startfilter = 48
#CNN network start layer, mid layers and lstm layer kernel size
start_kernel = 3
mid_kernel = 3
#Network depth has to be 9n + 2, n= 3 or 4 is optimal for Notum dataset
depth = 29
#Training epochs, longer the better with proper chosen learning rate
epochs = 150
#Size of the gradient descent length vector, start small and use callbacs to get smaller when reaching the minima
learning_rate = 1.0E-6
#For stochastic gradient decent, the batch size used for computing the gradients
batch_size = 10
# use softmax for single event per box, sigmoid for multi event per box
multievent = False
# Grid and number of boxes for yolo
gridX = 1
gridY = 1
Anchors = [0.11,0.11, 0.17,0.19, 0.26,0.29, 0.37,0.37, 0.62,0.55]
#Weightage to co-ordinate loss term
lambdacord = 1

In [4]:
config = StaticNeatConfig(residual = residual, depth = depth, start_kernel = start_kernel, mid_kernel = mid_kernel,
                         startfiler = startfilter, gridX = gridX, gridY = gridY, boxes = int(len(Anchors))/2, lambdacord = lambdacord,
                         epochs = epochs, categories = len(KeyCategories), box_vector = len(KeyCord), learning_rate = learning_rate,
                         batch_size = batch_size, ModelName = Model_Name, multievent = multievent)

config_json = config.to_json()
show = True
print(config)
save_json(config_json, ModelDir + Model_Name + '_Parameter.json')

StaticNeatConfig(ModelName='ORStaticONEAT.h5', batch_size=10, box_vector=5, categories=4, depth=29, epochs=150, gridX=1, gridY=1, lambdacord=1, learning_rate=1e-06, mid_kernel=3, multievent=False, nboxes=1, residual=True, start_kernel=3, startfilter=48)


In [5]:
Train = NEATStaticDetection(config, TrainDataDir, KeyCategories, KeyCord, ModelDir, Model_Name, show = show)

Train.loadData()

Train.TrainModel()


number of  images:	 1690
image size:		 (1690, 124, 124, 1)
Labels:				 (1690, 1, 1, 9)













Instructions for updating:
Use `tf.cast` instead.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 4 480         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 4 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, N

conv2d_26 (Conv2D)              (None, None, None, 3 295296      activation_22[0][0]              
__________________________________________________________________________________________________
batch_normalization_23 (BatchNo (None, None, None, 3 1536        conv2d_26[0][0]                  
__________________________________________________________________________________________________
activation_23 (Activation)      (None, None, None, 3 0           batch_normalization_23[0][0]     
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, None, None, 3 1327488     activation_23[0][0]              
__________________________________________________________________________________________________
batch_normalization_24 (BatchNo (None, None, None, 3 1536        conv2d_27[0][0]                  
__________________________________________________________________________________________________
activation

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Incompatible shapes: [10,1,2,2] vs. [10,1,1,4]
	 [[{{node loss/Concat_loss/mul_14}}]]
	 [[loss/add_32/_937]]
  (1) Invalid argument: Incompatible shapes: [10,1,2,2] vs. [10,1,1,4]
	 [[{{node loss/Concat_loss/mul_14}}]]
0 successful operations.
0 derived errors ignored.

<Figure size 432x288 with 0 Axes>